<a href="https://colab.research.google.com/github/yashv04/Batting-Analysis/blob/main/Top_Batsmen_against_CSK_Spin_in_Chepauk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max.colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
deliveries = pd.read_csv('deliveries.csv')
matches = pd.read_csv('matches.csv')

In [ ]:
df = deliveries.copy()

In [ ]:
df.head(2)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
mdf = matches.copy()

In [ ]:
mdf.head(1)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN


In [ ]:
mdf = mdf.rename(columns = {'id': 'match_id'})

In [ ]:
comb = pd.merge(df, mdf, on= 'match_id', how = 'left')

In [ ]:
comb.head(1)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN


In [ ]:
comb['over_no'] = comb['over'].apply(lambda x: x-1)

In [ ]:
def ByCustom(df, current_venue, current_phase, current_opposition):
    df = df[df['venue'] == current_venue]
    df = df[df['season'] == current_phase]
    df = df[df['bowling_team'] == current_opposition]
    df = df.reset_index(drop = True)
    return df

In [ ]:
def ByCustom(df, current_venue, current_phase, current_opposition):



    df['isDot']= df['batsman_runs'].apply(lambda x: 1 if x==0 else 0)
    df['isOne']= df['batsman_runs'].apply(lambda x: 1 if x==1 else 0)
    df['isTwo']= df['batsman_runs'].apply(lambda x: 1 if x==2 else 0)
    df['isThree']= df['batsman_runs'].apply(lambda x: 1 if x==3 else 0)
    df['isFour']= df['batsman_runs'].apply(lambda x: 1 if x==4 else 0)
    df['isSix']= df['batsman_runs'].apply(lambda x: 1 if x==6 else 0)

    runs = pd.DataFrame(df.groupby(['batsman', 'match_id'])['batsman_runs'].sum().reset_index()).groupby(['batsman'])['batsman_runs'].sum().reset_index().rename(columns={'batsman_runs':'runs'})
    innings = pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x)))).reset_index()).rename(columns = {'match_id':'innings'})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns = {'match_id':'balls'})
    dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns = {'player_dismissed':'dismissals'})


    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns = {'isDot':'dots'})
    twos = pd.DataFrame(df.groupby(['batsman'])['isTwo'].sum()).reset_index().rename(columns = {'isTwo':'twos'})
    threes = pd.DataFrame(df.groupby(['batsman'])['isThree'].sum()).reset_index().rename(columns = {'isThree':'threes'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'fours'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'sixes'})


    df = pd.merge(runs, balls, on = 'batsman').merge(innings, on = 'batsman').merge(dismissals, on = 'batsman').merge(sixes, on = 'batsman').merge(fours, on = 'batsman').merge(dots, on = 'batsman')

    df['balls']= df['balls'].astype(int)
    df['RPI']= df.apply(lambda x: (x['runs']/x['innings']) if x['innings'] != 0 else 0, axis = 1)
    df['SR']= df.apply(lambda x: 100*(x['runs']/x['balls']) if x['balls'] != 0 else 0, axis = 1)
    df['BPD']= df.apply(lambda x: (x['balls']/x['dismissals']) if x['dismissals'] != 0 else 0, axis = 1)
    df['BPB']= df.apply(lambda x: (x['balls']/(x['fours'] + x['sixes'])) if (x['fours'] + x['sixes']) != 0 else 0, axis = 1)
    df['dot_percentage']= df.apply(lambda x: 100*(x['dots']/x['balls']) if x['balls'] != 0 else 0, axis = 1)
    df['boundary_percentage'] = df.apply(lambda x: 100*((x['fours'] + x['sixes'])/x['balls']), axis = 1)

    return df

In [ ]:
def get_phase(over):
    if over <= 6:
        return 'powerplay'
    elif over < 16:
        return 'middle'
    else:
        return 'death'

In [ ]:
comb['phase'] = comb['over'].apply(lambda x: get_phase(x))

In [ ]:
df = ByCustom(comb, 'MA Chidambaram Stadium, Chepauk', 'middle', 'Chennai Super Kings')

In [ ]:
df.head(5)

,batsman,runs,balls,innings,dismissals,sixes,fours,dots,RPI,SR,BPD,BPB,dot_percentage,boundary_percentage
0,A Ashish Reddy,280,196,23,15,15,16,61,12.173913,142.857143,13.066667,6.322581,31.122449,15.816327
1,A Chandila,4,7,2,1,0,0,3,2.000000,57.142857,7.000000,0.000000,42.857143,0.000000
2,A Chopra,53,75,6,5,0,7,45,8.833333,70.666667,15.000000,10.714286,60.000000,9.333333
3,A Choudhary,25,20,3,2,1,1,4,8.333333,125.000000,10.000000,10.000000,20.000000,10.000000
4,A Flintoff,62,57,3,2,2,5,24,20.666667,108.771930,28.500000,8.142857,42.105263,12.280702


#SAATY SCALE

In [ ]:
import numpy as np

# Pairwise comparison matrix with the desired prioritization
pairwise_matrix = np.array([
    [1, 2, 3, 4, 5],              # RPI: Most important
    [0.5, 1, 2, 3, 4],            # SR: Second most important
    [0.33, 0.5, 1, 2, 3],         # BP: Third most important
    [0.25, 0.33, 0.5, 1, 2],      # DP: Fourth most important
    [0.2, 0.25, 0.33, 0.5, 1]     # BPD: Least important
])

# Normalize the matrix
normalized_matrix = pairwise_matrix / pairwise_matrix.sum(axis=0)
print(normalized_matrix)

[[0.43859649 0.49019608 0.43923865 0.38095238 0.33333333]
 [0.21929825 0.24509804 0.29282577 0.28571429 0.26666667]
 [0.14473684 0.12254902 0.14641288 0.19047619 0.2       ]
 [0.10964912 0.08088235 0.07320644 0.0952381  0.13333333]
 [0.0877193  0.06127451 0.04831625 0.04761905 0.06666667]]


In [ ]:
import numpy as np

# Your normalized matrix (from previous step)
normalized_matrix = np.array([
  [0.43859649, 0.49019608, 0.43923865, 0.38095238, 0.33333333],
  [0.21929825, 0.24509804, 0.29282577, 0.28571429, 0.26666667],
  [0.14473684, 0.12254902, 0.14641288, 0.19047619, 0.2       ],
  [0.10964912, 0.08088235, 0.07320644, 0.0952381,  0.13333333],
  [0.0877193,  0.06127451, 0.04831625, 0.04761905, 0.06666667]
])

# Calculate weights (average of each row)
weights = normalized_matrix.mean(axis=1)

wt_rpi, wt_sr, wt_bpd, wt_bp, wt_dot_percentage = weights    #defining wt as weights

print(weights)

[0.41646339 0.2619206  0.16083499 0.09846187 0.06231916]


In [ ]:
# Create weighted matrix
weighted_matrix = normalized_matrix * weights[:, np.newaxis]

# Print the weighted matrix
print(weighted_matrix)

[[0.18265938 0.20414872 0.18292682 0.15865272 0.13882113]
 [0.05743873 0.06419623 0.0766971  0.07483446 0.0698455 ]
 [0.02327875 0.01971017 0.02354831 0.03063524 0.032167  ]
 [0.01079626 0.00796383 0.00720804 0.00937732 0.01312825]
 [0.00546659 0.00381858 0.00301103 0.00296758 0.00415461]]


In [ ]:
#defining best and worst



df['calc_SR'] = df['SR'].apply(lambda x: x*x)
df['calc_RPI'] = df['RPI'].apply(lambda x: x*x)
df['calc_BPD'] = df['BPD'].apply(lambda x: x*x)
df['calc_dot_percentage'] = df['dot_percentage'].apply(lambda x: x*x)
df['calc_boundary_percentage'] = df['boundary_percentage'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage, sq_bp = np.sqrt(df[['calc_SR', 'calc_RPI', 'calc_BPD', 'calc_dot_percentage', 'calc_boundary_percentage']].sum())

df['calc_SR'] = df['calc_SR'].apply(lambda x: x/sq_sr)
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x/sq_rpi)
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x/sq_bpd)
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x/sq_dot_percentage)
df['calc_boundary_percentage'] = df['calc_boundary_percentage'].apply(lambda x: x/sq_bp)

df['calc_SR'] = df['calc_SR'].apply(lambda x: x*wt_sr)
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x*wt_rpi)
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x*wt_bpd)
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x*wt_dot_percentage)
df['calc_boundary_percentage'] = df['calc_boundary_percentage'].apply(lambda x: x*wt_bp)

best_sr, worst_sr = max(df['calc_SR']), min(df['calc_SR'])
best_rpi, worst_rpi = max(df['calc_RPI']), min(df['calc_RPI'])
best_bpd, worst_bpd = max(df['calc_BPD']), min(df['calc_BPD'])
best_dot_percentage, worst_dot_percentage = max(df['calc_dot_percentage']), min(df['calc_dot_percentage'])
best_bp, worst_bp = max(df['calc_boundary_percentage']), min(df['calc_boundary_percentage'])


df['dev_best_sr'] = df['calc_SR'].apply(lambda x: (x - best_sr)*(x - best_sr))
df['dev_best_rpi'] = df['calc_RPI'].apply(lambda x: (x - best_rpi)*(x - best_rpi))
df['dev_best_bpd'] = df['calc_BPD'].apply(lambda x: (x - best_bpd)*(x - best_bpd))
df['dev_best_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x - best_dot_percentage)*(x - best_dot_percentage))
df['dev_best_bp'] = df['calc_boundary_percentage'].apply(lambda x: (x - best_bp)*(x - best_bp))

# Calculate 'dev_best_sqrt' after creating the necessary columns
df['dev_best_sqrt'] = df['dev_best_sr'] + df['dev_best_rpi'] + df['dev_best_bpd'] + df['dev_best_dot_percentage'] + df['dev_best_bp']

df['dev_worst_sr'] = df['calc_SR'].apply(lambda x: (x - worst_sr)*(x - worst_sr))
df['dev_worst_rpi'] = df['calc_RPI'].apply(lambda x: (x - worst_rpi)*(x - worst_rpi))
df['dev_worst_bpd'] = df['calc_BPD'].apply(lambda x: (x - worst_bpd)*(x - worst_bpd))
df['dev_worst_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x - worst_dot_percentage)*(x - worst_dot_percentage))
df['dev_worst_bp'] = df['calc_boundary_percentage'].apply(lambda x: (x - worst_bp)*(x - worst_bp))

df['dev_worst_sqrt'] = df['dev_worst_sr'] + df['dev_worst_rpi'] + df['dev_worst_bpd'] + df['dev_worst_dot_percentage'] + df['dev_worst_bp']

In [ ]:
df['score']= df.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [ ]:
df = df[(df.balls > 20) & (df.innings > 2)]   #Considering batsmen who have played more than 2 innings and have faced minimum 20 balls.

In [ ]:
df[['batsman','runs', 'balls', 'SR', 'RPI', 'BPD', 'boundary_percentage', 'dot_percentage', 'score']].sort_values(by = 'score', ascending = False).head(30)

,batsman,runs,balls,SR,RPI,BPD,boundary_percentage,dot_percentage,score
24,AJ Tye,53,30,176.666667,17.666667,30.000000,23.333333,26.666667,0.567326
110,Iqbal Abdulla,88,86,102.325581,6.769231,86.000000,11.627907,41.860465,0.552112
48,BCJ Cutting,124,70,177.142857,15.500000,14.000000,27.142857,38.571429,0.521033
291,Shahid Afridi,81,46,176.086957,9.000000,5.750000,28.260870,43.478261,0.492182
150,LJ Wright,106,63,168.253968,21.200000,15.750000,30.158730,34.920635,0.489639
273,SN Khan,177,103,171.844660,16.090909,12.875000,24.271845,24.271845,0.487080
21,AD Russell,574,350,164.000000,22.960000,17.500000,24.285714,36.571429,0.470870
145,Kamran Akmal,128,78,164.102564,21.333333,19.500000,26.923077,42.307692,0.469886
60,CH Gayle,3651,2532,144.194313,36.510000,28.133333,22.235387,46.524487,0.461324
67,CR Brathwaite,95,56,169.642857,11.875000,7.000000,28.571429,48.214286,0.458615


#Decoding Middle-Overs Mastery: Top Batsmen against CSK Spin in Chepauk
#Chepauk, the spin-friendly fortress of the Chennai Super Kings (CSK), presents a unique challenge for batsmen during the middle overs. While CSK's spinners weave their magic, a select few batsmen have emerged as spin wizards, defying the odds with their calculated aggression and unwavering consistency.
#Our data-driven quest to uncover these masters led us to analyze key metrics like Strike Rate (SR), Runs Per Innings (RPI), Balls Per Dismissal (BPD), Boundary Percentage (BP), and Dot Percentage (DP). The results reveal a compelling narrative of batsmen who have not only survived but thrived in this challenging environment.
#These players have stand out as the spin conquerors of Chepauk. Their high SR, RPI and BP showcase their ability to pierce the field and find boundaries against turning deliveries. Their impressive RPI and BPD further highlight their consistency and ability to build innings, making them invaluable assets during the middle overs.
#These spin wizards have mastered the art of tackling spin in Chepauk, decoding the code, finding the gaps, and executing their game plans with precision. Their ability to score quickly, find boundaries, and maintain consistency makes them the preferred choices for any team seeking to dominate the middle overs against CSK in Chennai. As the IPL continues, these names will be the ones to watch – the spin conquerors of Chepauk.